In [16]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.graph_objs as go
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from crud_module import CRUD

###########################
# Data Manipulation / Model
###########################
host = 'nv-desktop-services.apporto.com'
port = 31738
database = 'AAC'
authentication_database = 'admin'
collection = 'animals'
username = 'aacuser'
password = 'SNHU1234'

db = CRUD(host, port, database, authentication_database, collection, username, password)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.query_documents({}))

df['rescue_type'] = df['rescue_type'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id', 'Preferred Breeds', 'Preferred Sex', 'Training Age'], inplace=True)

# Columns to be displayed in the data table and their order
table_columns = [
    "name", "animal_type", "breed", "sex_upon_outcome", "color", "age_upon_outcome",
    "age_upon_outcome_in_weeks", "date_of_birth", "datetime", "monthyear", "outcome_type",
    "outcome_subtype", "rescue_type", "rec_num", "animal_id", 'location_lat', 'location_long'
]

df_display = df[table_columns]

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Limit the number of animals displayed in the data table per page
PAGE_SIZE = 25

#########################
# Dashboard Layout / View
#########################
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.H2('Anthony Premo')),
    html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '100px'}),
           href='https://www.snhu.edu'),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water', 'value': 'Water'},
                {'label': 'Mountain or Wilderness', 'value': 'Mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_display.columns],
        data=df_display.to_dict('records'),
        page_size=25
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'width': '100%', 'height': '500px'}, children=[
        html.Div(
            id='pie-chart-id',
            className='col s12 m6', style={'float': 'left', 'width': '50%', 'height': '100%'}
        ),
        html.Div(
            id='map-id',
            className='col s12 m6', style={'float': 'right', 'width': '50%', 'height': '100%'}
            
        ),
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_data_table(filter_type):
    if filter_type == 'Reset':
        # If Reset is selected, show all data in the data table
        data_filtered = df.to_dict('records')
    else:
        # Apply filter based on the selected rescue type
        data_filtered = df[df['rescue_type'].apply(lambda x: filter_type in x if isinstance(x, str) else filter_type is None)].to_dict('records')
    return data_filtered


@app.callback(
    Output('pie-chart-id', "children"),
    [Input('datatable-id', "data")])
def update_pie_chart(data):
    if data is None:
        return []
    df_filtered = pd.DataFrame.from_dict(data)
    if df_filtered.empty:
        return []
    # Calculate the percentage of animals for each rescue type
    rescue_type_counts = df_filtered['rescue_type'].value_counts(normalize=True)
    # Create a pie chart to display the percentage of animals for each rescue type
    fig = go.Figure(data=[go.Pie(labels=rescue_type_counts.index, values=rescue_type_counts.values)])
    fig.update_layout(title='Percentage of Animals by Rescue Type')
    return [
        dcc.Graph(figure=fig)
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # Check if there is data in the data table
    if viewData is not None:
        # Create animal markers for each row in the data table
        markers = []
        for row in viewData:
            lat = row.get('location_lat')
            lon = row.get('location_long')
            if lat is not None and lon is not None:
                marker = dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(row['name']),
                        dl.Popup([
                            html.H6(row['name']),
                            html.P(f"Animal Type: {row['animal_type']}"),
                            html.P(f"Breed: {row['breed']}"),
                            # Add more information about the animal if desired
                        ])
                    ]
                )
                markers.append(marker)

        # Create an empty map with the TileLayer only
        map_children = dl.Map(
            style={'width': '100%', 'height': '100%'},
            center=[30.0, -97.0],  # Default center if no data or no rows selected
            zoom=8,
            children=[
                dl.TileLayer(id="base-layer-id")
            ]
        )

        # Add animal markers to the map
        map_children.children.extend(markers)

        return map_children

    # If there is no data, return an empty map with the TileLayer only
    return dl.Map(
        style={'width': '100%', 'height': '100%'},
        center=[30.0, -97.0],  # Default center if no data or no rows selected
        zoom=8,
        children=[
            dl.TileLayer(id="base-layer-id")
        ]
    )

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:19913/
